In [25]:
import pandas as pd
import datetime
import numpy as np
import gym
from gym import spaces

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

# Building the Traditional Model

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, RobustScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier


In [19]:
df = pd.read_csv('Fraud_Data.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151112 entries, 0 to 151111
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user_id         151112 non-null  int64  
 1   signup_time     151112 non-null  object 
 2   purchase_time   151112 non-null  object 
 3   purchase_value  151112 non-null  int64  
 4   device_id       151112 non-null  object 
 5   source          151112 non-null  object 
 6   browser         151112 non-null  object 
 7   sex             151112 non-null  object 
 8   age             151112 non-null  int64  
 9   ip_address      151112 non-null  float64
 10  class           151112 non-null  int64  
dtypes: float64(1), int64(4), object(6)
memory usage: 12.7+ MB


In [20]:
df.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0


In [21]:
# feature engineering

df['signup_date'] = pd.to_datetime(df['signup_time'])
df['purchase_time'] = pd.to_datetime(df['purchase_time'])
df['time_Since_signup'] = (df['purchase_time'] - df['signup_date']).dt.total_seconds()
df['time_of_day'] =df['purchase_time'].dt.hour

In [22]:
df.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,signup_date,time_Since_signup,time_of_day
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,2015-02-24 22:55:49,4506682.0,2
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,2015-06-07 20:39:50,17944.0,1
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,2015-01-01 18:52:44,1.0,18
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,2015-04-28 21:13:25,492085.0,13
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,2015-07-21 07:09:52,4361461.0,18


In [28]:
df['source'].unique()

array(['SEO', 'Ads', 'Direct'], dtype=object)

In [36]:
#select features for state representation

features = ['source', 'time_Since_signup', 'time_of_day', 'age', 'sex', 'device_id']

X = df[features]
y = df['class']


ohe = OneHotEncoder()
scaler = RobustScaler()

cat_cols = ['sex', 'device_id', 'source']
num_col = ['age', 'time_Since_signup', 'time_of_day']


preprocessor = ColumnTransformer(
    transformers=[
        ('num', scaler, num_col),
        ('cat', ohe, cat_cols)
    ]
)

X_preprocessed = preprocessor.fit_transform(X)

In [37]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_preprocessed, y)

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [43]:
import gym
from gym import Env
from gym.spaces import Discrete, Box


class FraudDetection(Env):
    def __init__(self, data, labels):
        super(FraudDetection, self).__init__()
        self.data = data
        self.labels = labels
        self.current_index = 0
        self.action_space = Discrete(3)

        self.observation_space = Box(low=-np.inf, high=np.inf, shape=(data.shape[1],), dtype=np.float32)

    def reset(self):
        self.current_index = 0
        return self.data[self.current_index]
    
    def step(self, action):
        actual_label = self.labels[self.current_index]
        reward = get_reward(actual_label, action)
        self.current_index += 1

        done = self.current_index >= len(self.data)

        next_state = self.data[self.current_index] if not done else None

        return next_state, reward, done, {}
    

# train env
env = FraudDetection(X_train, y_train)

In [45]:
# train and test

from stable_baselines3 import DQN
from stable_baseline3.common.env_util import make_vec_env

env = make_vec_env(lambda: FraudDetection(X_train, y_train), n_envs=1)

model = DQN("MlpPolicy", env, learning_rate = 0.0005, verbose=1)

model.learn(total_timesteps=10)

ModuleNotFoundError: No module named 'stable_baseline3'